## MyHomeBudget

Aplikacja MyFinance wykorzystuje `TabLayout Navigation` z `ViewPager2`. Dane dostarczymy przez klasę `dataProvider` zawierający hardcodowane przykładowe pozycje (*dummy data*). Aplikacja zawiera trzy ekrany pokazujące stan oszczędności oraz rachunki - ekran główny zawiera podsumowanie. Dodamy customową animację do `TabLayout.Tab` oraz `DonutChart` podsumowujący stan naszych finansów oraz kilka elementów urozmaicających wygląd naszej aplikacji. Sam layout aplikacji bazuje na przykładzie z **Android Open Project** [link](https://github.com/android/compose-samples/tree/main/Rally)

<table><tr><td><img src="https://media1.giphy.com/media/49RzyXBmRBa59Wi47n/giphy.gif?cid=790b76111b9b9991bd4197bf8d7295e517603d04f8ba2992&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media1.giphy.com/media/zbqykvBYncTRQLCAja/giphy.gif?cid=790b7611d47e3d6d46e7b736ac29fe3cc9bc281aed07c1c0&rid=giphy.gif&ct=g" width="200" /></td><td><img src="https://media3.giphy.com/media/BbcDIUyV6IodX2yGVx/giphy.gif?cid=790b7611d1bf2abf7deacc8ad5a71a6a13adfbc9f4344954&rid=giphy.gif&ct=g" width="200" /></td></tr></table>

Rozpocznijmy od dodania zależności do projektu - wykorzystamy `DonutChart` (wykorzystamy [doughnut](https://github.com/futuredapp/donut)) oraz `CardView`

In [ ]:
implementation("app.futured.donut:donut:2.2.2")
implementation ("androidx.cardview:cardview:1.0.0")

Tak jak w poprzednich aplikacjach, będdziemy wykorzystywać `ViewBinding`

In [ ]:
android {
    ...
    buildFeatures {
        viewBinding = true
    }
}

Nasza aplikacja będzie posiadała zdefiniowany layout tylko w orientacji `portrait`, więc zablookujemy możliwość zmiany w plik `AndroidManifest.xml` - do aktywności dodamy

In [ ]:
android:screenOrientation="portrait"

### **TabLayout i ViewPager2**

Rozpoczniemy od dodania nawigacji. Aplikacja będzie posiadać trzy ekrany, więc dodajmy trzy fragmenty (`BlankFragment`) - `OverviewFragment`, `AccountsFragment`, `BillsFragment`

In [ ]:
public class AccountsFragment extends Fragment {

    private FragmentAccountsBinding binding;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentAccountsBinding.inflate(inflater);
        return binding.getRoot();
    }
}

In [ ]:
public class BillsFragment extends Fragment {

    private FragmentBillsBinding binding;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentBillsBinding.inflate(inflater);
        return binding.getRoot();
    }
}

In [ ]:
public class OverviewFragment extends Fragment {

    private FragmentOverviewBinding binding;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentOverviewBinding.inflate(inflater);
        return binding.getRoot();
    }
}

W layoutach chwilowo zmieniam tylko text pola `TextView` na nazwę fragmentu. Ponieważ nasza aplikacja będzie posiadać tylko jeden motyw usuwamy plik `themes.xml(night)` do pliku `colors.xml` dodajemy kilka kolorów

In [ ]:
<color name="green_500">#FF1EB980</color>
<color name="dark_blue_900">#FF26282F</color>
<color name="dark_blue_500">#FF004940</color>

Następnie modyfikujemy plik `themes.xml`

In [ ]:
<resources xmlns:tools="http://schemas.android.com/tools">
    <!-- Base application theme. -->
    <style name="Theme.MyHomeBudgetKotlin" parent="Theme.MaterialComponents.DayNight.NoActionBar">
        <!-- Primary brand color. -->
        <item name="colorPrimary">@color/dark_blue_500</item>
        <item name="colorPrimaryVariant">@color/dark_blue_900</item>
        <item name="colorOnPrimary">@color/teal_700</item>
        <!-- Secondary brand color. -->
        <item name="colorSecondary">@color/teal_700</item>
        <item name="colorSecondaryVariant">@color/teal_200</item>
        <item name="colorOnSecondary">@color/white</item>
        <!-- Status bar color. -->
        <item name="android:statusBarColor" >?attr/colorPrimaryVariant</item>
        <!-- Customize your theme here. -->
    </style>
</resources>

Aplikacja nie będzie wykorzystywać `ActionBar`, więc jako `parent` podajemy `NoActionBar`.

Nasz `ViewPager2` dodamy bezpośrednio do `MainActivity` - zmodyfikujmy layout

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:orientation="vertical">

    <com.google.android.material.tabs.TabLayout
        android:id="@+id/tabLayout"
        android:layout_width="match_parent"
        android:background="@color/dark_blue_900"
        app:tabTextColor="@color/white"
        android:layout_height="wrap_content"
        app:tabInlineLabel="true"
        app:tabIconTint="@color/teal_200"
        app:tabMode="fixed"
        app:tabGravity="start"
        app:tabMaxWidth="0dp" />

    <androidx.viewpager2.widget.ViewPager2
        android:id="@+id/viewPager"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"/>

</LinearLayout>

Kilka elementów zastosowanych w `TabLayout`
- `tabMode` - sposób w jaki mają zostać wyświetlone zakładki
    - `fixed` - tak jak zostaną podane, rozmiar zostanie automatycznie dostosowany
    - `scrollable` - zakładki nie będą dokowane na całą szerokość ekranu, tylko kilka zostanie wyświetlonych - dostęp do niewidocznych użytkownik może uzyskać przez przewijanie
- `tabGravity` - sposób rozmieszczenia zakładek
- `tabMaxWidth` - ustawienie maksymalnej szerokości - `0dp` dostosowuje rozmiar zakładki do jej zawartości
- `tabIconTint` - kolor wyświetlanej ikony
- `tabInlineLabel` - ustawienie ikony i tekstu w rzędzie - przyjmuje `true` lub `false`

`ViewPager2` zawiera `layout_height` ustawiony na `0dp` oraz `layout_weight = "1"` pozwala wypełnić pozostałą przestrzeń w `LinearLayout`

Dodajmy klasę `FinanceAdapter` będącą adapterem naszego `ViewPager2`

In [ ]:
class FinanceAdapter extends FragmentStateAdapter {

    private final Fragment[] fragments = {
            new OverviewFragment(),
            new AccountsFragment(),
            new BillsFragment()
    };

    public FinanceAdapter(@NonNull FragmentActivity fragmentActivity) {
        super(fragmentActivity);
    }

    @NonNull
    @Override
    public Fragment createFragment(int position) {
        return fragments[position];
    }

    @Override
    public int getItemCount() {
        return fragments.length;
    }
}

W klasie definiujemy listę fragmentów która jest inicjalizowana trzema fragmentami: `OverviewFragment`, `AccountsFragment` i `BillsFragment` - są to komponenty naszej nawigacji.

Następnie utworzymy `ViewPager2` i `TabLayout` w metodzie `onCreate` głównej aktywności

In [ ]:
public class MainActivity extends AppCompatActivity {

    private ActivityMainBinding binding;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());


        binding.viewPager.setAdapter(new FinanceAdapter(this));

    }
}

Następnym krokiem jest napisanie metody `setupTabLayoutMediator`, dodajmy do projektu nowy pakiet `util` i w nim utworzymy nowy plik `TabSetupUtil`. Do pliku dodajmy tytuły naszych zakładek

In [ ]:
public final class TabSetupUtil {
    private TabSetupUtil(){}

    private static final int[] tabTitles = {R.string.overview, R.string.accounts, R.string.bills};
}

I do pliku `strings.xml`

In [ ]:
<string name="overview">Overview</string>
<string name="accounts">Accounts</string>
<string name="bills">Bills</string>

Dodajmy metodę `setupTabLayoutMediator` do pliku `UiSetup`

In [ ]:
public static void setupTabLayoutMediator(
        Context context,
        TabLayout tabLayout,
        ViewPager2 viewPager2
) {
    new TabLayoutMediator(tabLayout, viewPager2,
            (tab, position) -> {
                tab.setText(context.getString(tabTitles[position]));
            }
    ).attach();
}

In [ ]:
public class MainActivity extends AppCompatActivity {

    private ActivityMainBinding binding;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());


        binding.viewPager.setAdapter(new FinanceAdapter(this));
        TabSetupUtil.setupTabLayoutMediator(this, binding.tabLayout, binding.viewPager);
    }
}

Jest to funkcja, która łączy `TabLayout` z `ViewPager2`.

`TabLayoutMediator` to klasa, która umożliwia połączenie `TabLayout` i `ViewPager2`. Następnie używamy lambdy, aby ustawiać tekst w każdym elemencie `TabLayout`. Wewnętrzna zmienna `position` przechowuje pozycję aktualnego fragmentu, a `context.getString(tabTitles[position])` zwraca tekst, który jest pobierany z tablicy stringów o nazwie `tabTitles` zdefiniowanej wcześniej.

Na koniec wywołujemy metodę `attach()`, aby zakończyć proces połączenia `TabLayout` i `ViewPager2`.

Możemy przetestować aplikację

<img src="https://media1.giphy.com/media/9IdIFNbyyewccfTTjG/giphy.gif?cid=790b76113f11b618eb6ce26cdf2e17faf07d98203cdf7e21&rid=giphy.gif&ct=g" width="150" />

Dodajmy zestaw ikon dla naszych zakładek - przechodzimy do pliku `uiSetup`

In [ ]:
private static final int[] tabTitles = {R.string.overview, R.string.accounts, R.string.bills};
private static final int[] tabIcons = {
    R.drawable.ic_overview,
    R.drawable.ic_accounts, 
    R.drawable.ic_bills};

Zmodyfikujmy metodę `setupTabLayoutMediator`

In [ ]:
public static void tabLayoutSetup(Context context, TabLayout tabLayout, ViewPager2 viewPager) {
    setupTabMediator(context, tabLayout, viewPager);
}

private static void setupTabMediator(
        Context context,
        TabLayout tabLayout,
        ViewPager2 viewPager ){
    new TabLayoutMediator(tabLayout, viewPager,
            (tab, position) -> {
                tab.setIcon(ContextCompat.getDrawable(context, tabIcons[position]));
                tab.setText(context.getString(tabTitles[position]));
                Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
            }
    ).attach();
}

Metoda `Objects.requireNonNull(tab.getIcon()).setTint()` pozwala zmienić kolor grafiki wektorowej - ustawiamy na białą. Nasza aplikacja teraz wygląda następująco.

<img src="https://media0.giphy.com/media/gGFILN8lnvSsCIacNt/giphy.gif?cid=790b7611d2de0256b614247faddd193614ba76fa00201eec&rid=giphy.gif&ct=g" width="150" />

Chcemy pokazać tylko ikonę nie zaznaczonej zakładki oraz ikonę z tekstem wybranej zakładce. Dodajmy funkcję `setupTabSelection` w której zaimplementujemy metody `addOnTabSelectedListener`, oraz `tabLayoutSetup`, którą wywołamy w `MainActivity`. 

`TabSelectionListener` obsługuje tylko zdarzenia `onTabSelected`, `onTabUnselected` i `onTabReselected`. W pierwszych dwóch zmienimy tekst zakładki

In [ ]:
public final class TabSetupUtil {
    private TabSetupUtil(){}

    private static final int[] tabTitles = {R.string.overview, R.string.accounts, R.string.bills};
    private static final int[] tabIcons = {
            R.drawable.ic_overview,
            R.drawable.ic_accounts,
            R.drawable.ic_bills};

    public static void tabLayoutSetup(Context context, TabLayout tabLayout, ViewPager2 viewPager) {
        setupTabMediator(context, tabLayout, viewPager);
        setupTabSelection(context, tabLayout);
    }

    private static void setupTabMediator(
            Context context,
            TabLayout tabLayout,
            ViewPager2 viewPager ){
        new TabLayoutMediator(tabLayout, viewPager,
                (tab, position) -> {
                    tab.setIcon(ContextCompat.getDrawable(context, tabIcons[position]));
                    tab.setText(context.getString(tabTitles[position]));
                    Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
                }
        ).attach();
    }

    private static void setupTabSelection(Context context, TabLayout tabLayout) {
        tabLayout.addOnTabSelectedListener(new TabLayout.OnTabSelectedListener() {
            @Override
            public void onTabSelected(TabLayout.Tab tab) {
                tab.setText(context.getString(tabTitles[tab.getPosition()]));
                Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
            }

            @Override
            public void onTabUnselected(TabLayout.Tab tab) {
                tab.setText("");
            }

            @Override
            public void onTabReselected(TabLayout.Tab tab) {

            }
        });
    }
}

Metoda `addOnTabSelectedListener` pozwala na zdefiniowanie zachowań, które będą wykonywane po wybraniu, odznaczeniu lub ponownym wybraniu zakładki.

W przypadku tego kodu, gdy użytkownik wybierze zakładkę (`onTabSelected`), zmieniana jest wartość tekstu zakładki na wartość zdefiniowaną w `tabTitles` dla wybranej pozycji. Gdy użytkownik odznaczy zakładkę (`onTabUnselected`), wartość tekstu zostaje wyczyszczona. W przypadku ponownego wybrania zakładki (`onTabReselected`), nie dzieje się nic.

Podsumowując, ten kod pozwala na dynamiczne ustawianie tekstu na zakładkach w `TabLayout` na podstawie wartości zdefiniowanych w `tabTitles`.

In [ ]:
public class MainActivity extends AppCompatActivity {

    private ActivityMainBinding binding;

    @Override
    protected void onCreate(Bundle savedInstanceState) {
        super.onCreate(savedInstanceState);
        binding = ActivityMainBinding.inflate(getLayoutInflater());
        setContentView(binding.getRoot());


        binding.viewPager.setAdapter(new FinanceAdapter(this));
        TabSetupUtil.tabLayoutSetup(this, binding.tabLayout, binding.viewPager);
    }
}

<img src="https://media0.giphy.com/media/uzc9KIxaD4KinhS1GQ/giphy.gif?cid=790b7611153c63bbec947cab1bc09d6c638f7323bdc36e8c&rid=giphy.gif&ct=g" width="150" />

Musimy jeszcze zainicjować w odpowiedni sposób zakładki. Zmodyfikujmy metodę `setupTabLayoutMediator`. Tekst i kolor ikony ustawimy tylko dla `position == 0`, co odpowiada ekranowi głównemu aplikacji.

In [ ]:
private static void setupTabMediator(
        Context context,
        TabLayout tabLayout,
        ViewPager2 viewPager ){
    new TabLayoutMediator(tabLayout, viewPager,
            (tab, position) -> {
                tab.setIcon(ContextCompat.getDrawable(context, tabIcons[position]));
                if (position == 0) {
                    tab.setText(context.getString(tabTitles[position]));
                    Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);
                }
            }
    ).attach();
}

<img src="https://media0.giphy.com/media/8wosma6sipcea16wxt/giphy.gif?cid=790b76114eedb95754c5e7f3b59ff15167d82c6bc0910831&rid=giphy.gif&ct=g" width="150" />

### Animacje

Dodajmy customową animację do zakładek. Użyjemy prostego skalowania - niezaznaczona zakładka będzie wyświetlać tylko przeskalowaną (70%) ikonę. Dodajmy metodę `initialTabsSetup`

In [ ]:
private static final float tabScaleLow = 0.7f;
private static final float tabScaleHigh = 1f;

private static void initialTabsSetup(TabLayout tabLayout) {
    ViewGroup viewGroup = (ViewGroup) tabLayout.getChildAt(0);
    for (int i = 0; i < viewGroup.getChildCount(); i++) {
        if (i == tabLayout.getSelectedTabPosition())
            continue;
        View tab = viewGroup.getChildAt(i);
        tab.setScaleX(tabScaleLow);
        tab.setScaleY(tabScaleLow);
    }
}

Skalować możemy tylko na obiektach `View` - bezpośrednio nie możemy tego zrobić na `TabLayout.Tab`, ponieważ nie rozszerza on `View`. Więc jako parametr metody podamy `ViewGroup`, który będzie naszym zawierał nasze zakładki. Z `TabLayout` wywołujemy metodę `getChildAt()` wyciągając wszyskie zakładki przypisane do tego `TabLayout` i rzutujemy na `ViewGroup`. Chcemy skalować wszystkie elementy poza aktualnie zaznaczonym, dlatego mamy warunek `if (...) continue`. Następnie wywołajmy tą metodę w `tabLayoutSetup`

In [ ]:
public static void tabLayoutSetup(Context context, TabLayout tabLayout, ViewPager2 viewPager) {
    setupTabMediator(context, tabLayout, viewPager);
    initialTabsSetup(tabLayout);
    setupTabSelection(context, tabLayout);
}

Dodajmy metodę `setupAnimation` wykonującą prostą animację powiększania.

In [ ]:
private static void setupAnimation(View view, float scale, long duration) {
    view.animate()
            .scaleX(scale)
            .scaleY(scale)
            .setInterpolator(new FastOutSlowInInterpolator())
            .setDuration(duration)
            .start();
}

- `scaleX`, `scaleY` - skalowanie do zadanej wartości z przedziału (0.0 - 1.0)
- `setInterpolator` - umożliwia wykorzystanie podstawowych animacji
- `FastOutSlowInInterpolator` - wykorzysuje [tablicowanie](https://pl.wikipedia.org/wiki/Tablicowanie) z [krzywymi Beziera](https://pl.wikipedia.org/wiki/Krzywa_B%C3%A9ziera) do wykonania utworzenia animacji - tutaj zastosujemy skalowanie
- `setDuration` - długość animacji w milisekundach

Teraz wykorzystajmy tą metodę do zmiany rozmiarów zakładek po zaznaczeniu.

In [ ]:
private static void setupTabSelection(Context context, TabLayout tabLayout) {

    ViewGroup viewGroup = (ViewGroup) tabLayout.getChildAt(0);

    tabLayout.addOnTabSelectedListener(new TabLayout.OnTabSelectedListener() {
        @Override
        public void onTabSelected(TabLayout.Tab tab) {
            tab.setText(context.getString(tabTitles[tab.getPosition()]));
            Objects.requireNonNull(tab.getIcon()).setTint(Color.WHITE);

            // zwracam aktualną zakładkę jako View
            View currentChild = viewGroup.getChildAt(tab.getPosition());

            // czas animacji pobrany z zasobów systemowych Android
            long duration = (long) context.getResources()
                    .getInteger(android.R.integer.config_mediumAnimTime);

            setupAnimation(currentChild, tabScaleHigh, duration);
        }

        @Override
        public void onTabUnselected(TabLayout.Tab tab) {
            tab.setText("");

            View previousChild = viewGroup.getChildAt(tab.getPosition());
            long duration = 1L;

            setupAnimation(previousChild, tabScaleLow, duration);
        }

        @Override
        public void onTabReselected(TabLayout.Tab tab) {

        }
    });
}

private static void setupAnimation(View view, float scale, long duration) {
    view.animate()
            .scaleX(scale)
            .scaleY(scale)
            .setInterpolator(new FastOutSlowInInterpolator())
            .setDuration(duration)
            .start();
}

Podsumowując, konfigurujemy `TabLayout`. `tabLayoutSetup` jest funkcją, która przyjmuje trzy parametry: `context`, `tabLayout` i `viewPager2`. Wywołuje ona dwie funkcje pomocnicze: `setupTabLayoutMediator` oraz `setupTabSelection`.

`setupTabLayoutMediator` konfiguruje połączenie między `tabLayout` a `viewPager2`. W tym celu tworzona jest instancja klasy `TabLayoutMediator`. Następnie, dla każdej zakładki, ustawiany jest tekst oraz ikona, a także kolor tekstu. Ikony są pobierane z zasobów aplikacji.

Funkcja `setupTabSelection` ustawia zachowanie kart w momencie, gdy zostaną wybrane, odznaczone lub wybrane ponownie. Gdy karta zostaje wybrana, ustawiana jest nowa wartość tekstu, kolor tekstu, ikony, oraz wykonuje się animacja powiększania karty. Gdy karta zostaje odznaczona, tekst karty zostaje wyczyszczony, a karta zostaje zmniejszona.

Funkcja `initialTabsSetup` ustawia skalowanie kart na początku działania aplikacji. Iteruje po wszystkich dzieciach przekazanego `viewGroup` (który powinien być pierwszym dzieckiem `TabLayout` (`ViewGroup viewGroup = (ViewGroup) tabLayout.getChildAt(0)` w metodzie `tabLayoutSetup`), z wyjątkiem obecnie zaznaczonej zakładki, i ustawia ich skalę na wartość `tabScaleLow`.

Funkcja `setupAnimation` to funkcja pomocnicza, która tworzy animację powiększenia karty. Animuje skalowanie danego widoku `View`. Wewnątrz funkcji wywoływana jest metoda `animate()`, która inicjuje animację, a następnie metodami `scaleX()` i `scaleY()` ustawia docelowe skalowanie wraz z interpolatorem animacji (`FastOutSlowInInterpolator()`) i czasem trwania. Ostatecznie, animacja jest uruchamiana przez wywołanie metody `start()`.

<img src="https://media2.giphy.com/media/4sMOjoEDLmVGtiC7FD/giphy.gif?cid=790b761140e3aa2701d05ed0f4d943a6f3bf656a58a60e90&rid=giphy.gif&ct=g" width="150" />

### **Dane**

Dodajmy dane (*dummy data*) do naszego projektu. Rozpocznijmy od modelu - posłużymy się dwiema klasami danych.

Klasa `Account` zawiera pola
- `name` - nazwa konta
- `number` - numer rachunku
- `amount` - zebrana kwota
- `color` - kolor powiązany z kontem

Klasa `Bills`:
- `name` - nazwa rachunku
- `endDate` - data spłaty
- `amount` - kwota do spłaty
- `color` - kolor powiązany z rachunkiem

In [ ]:
public class Account {
    private final String name;
    private final String number;
    private final double amount;
    private final int color;

    public Account(String name, String number, double amount, int color) {
        this.name = name;
        this.number = number;
        this.amount = amount;
        this.color = color;
    }

    public String getName() {
        return name;
    }

    public String getNumber() {
        return number;
    }

    public double getAmount() {
        return amount;
    }

    public int getColor() {
        return color;
    }
}

In [ ]:
import java.time.LocalDate;

public class Bill {
    private final String name;
    private final LocalDate date;
    private final double amount;
    private final int color;

    public Bill(String name, LocalDate date, double amount, int color) {
        this.name = name;
        this.date = date;
        this.amount = amount;
        this.color = color;
    }

    public String getName() {
        return name;
    }

    public LocalDate getDate() {
        return date;
    }

    public double getAmount() {
        return amount;
    }

    public int getColor() {
        return color;
    }
}

Następnie dodajmy obiekt `DataProvider` udostępniający *dummy data*, które wykorzystamy w aplikacji. Każdy typ konta, oraz rachunków, będzie posiadał swój własny kolor.

In [ ]:
public final class DataProvider {
    private DataProvider() {}

    public static Account[] accounts = {
            new Account("Home savings", "1111111111", 23456.34, Color.BLUE),
            new Account("Car savings", "2222222222", 126578.99, Color.LTGRAY),
            new Account("Vacation", "3457733323", 9875.12, Color.MAGENTA),
            new Account("Emergency", "9488344443", 10000.77, Color.RED),
            new Account("Healthcare", "3243554434", 12345.00, Color.YELLOW),
            new Account("Shopping", "2947560007", 3456.56, Color.BLACK)
    };

    public static Bill[] bills = {
            new Bill("Bank Credit", LocalDate.of(2022, 9,22), 2300.0, Color.BLACK),
            new Bill("Tuition", LocalDate.of(2023, 2,10), 1200.0, Color.BLUE),
            new Bill("Rent", LocalDate.of(2022, 8,3), 1023.87, Color.YELLOW),
            new Bill("Loan", LocalDate.of(2022, 12,22), 334.0, Color.GRAY),
            new Bill("Car Repair", LocalDate.of(2023, 1,9), 982.33, Color.WHITE),
            new Bill("Dress Loan", LocalDate.of(2023, 5,18), 243.0, Color.MAGENTA)
    };

    public static double totalAccountsAmount = Arrays
            .stream(accounts)
            .map(Account::getAmount)
            .collect(Collectors.toList())
            .stream()
            .mapToDouble(Double::doubleValue)
            .sum();

    public static double totalBillsAmount = Arrays
            .stream(bills)
            .map(Bill::getAmount)
            .collect(Collectors.toList())
            .stream()
            .mapToDouble(Double::doubleValue)
            .sum();
}

W obiekcie `DataProvider` znnajdują się:
- `accounts` - lista wszystkich kont
- `bills` - lista wszystkich rachunków
- `totalAccountsAmount` - całkowita kwota na wszystkich rachunkach
- `totalBillsAmount` - całkowita kwota wszystkich rachunków

### **AccountsFragment**

Ponieważ `AccountsFragment` i `BillsFragment` będą używać tego samego layoutu, zdefiniujemy jeden plik `xml` o nazwie `fragment_detail_info`, rozpocznijmy od dodania `RecyclerView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900">
    
        <androidx.recyclerview.widget.RecyclerView
            android:id="@+id/recyclerView"
            android:layout_width="match_parent"
            android:layout_height="0dp"
            android:layout_marginTop="12dp"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintTop_toTopOf="parent" />
        
</androidx.constraintlayout.widget.ConstraintLayout>

Utwórzmy layout elementu `RecyclerView`

`recyclerview_item_view.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="wrap_content"
    android:layout_margin="4dp"
    android:background="@color/dark_blue_500">

    <View
        android:id="@+id/RVcolorBarView"
        android:layout_width="12px"
        android:layout_height="0dp"
        android:background="@color/white"
        android:layout_marginStart="8dp"
        android:layout_marginBottom="8dp"
        android:layout_marginTop="8dp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/RVNumberTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:fontFamily="serif-monospace"
        android:padding="4dp"
        android:text="******1111"
        android:textColor="@color/white"
        android:textSize="16sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toBottomOf="@+id/RVNameTextView" />

    <TextView
        android:id="@+id/RVNameTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:layout_marginStart="16dp"
        android:layout_marginTop="8dp"
        android:fontFamily="sans-serif-smallcaps"
        android:padding="4dp"
        android:text="Savings"
        android:textColor="@color/white"
        android:textSize="18sp"
        android:textStyle="bold"
        app:layout_constraintStart_toStartOf="@id/RVcolorBarView"
        app:layout_constraintTop_toTopOf="parent" />

    <TextView
        android:id="@+id/RVValueTextView"
        android:layout_width="wrap_content"
        android:layout_height="wrap_content"
        android:padding="16dp"
        android:text="1111111 zł"
        android:textColor="@color/white"
        android:textSize="30sp"
        app:layout_constraintBottom_toBottomOf="parent"
        app:layout_constraintEnd_toEndOf="parent"
        app:layout_constraintTop_toTopOf="parent" />

</androidx.constraintlayout.widget.ConstraintLayout>

Pierwszym elementem jest `View`, będzie to pasek pokazujący kolor powiązany w danym kontem.

Będziemy wyświetlać kwoty, więc dodajmy odpowiednie formatowanie, do pakietu `utils` dodajmy plik `FormatterUtil`. W nim dodajmy jedno pole zawierające `NumberFormat`

In [ ]:
public final class FormatterUtil {
    private FormatterUtil(){}

    public static NumberFormat formatter = new DecimalFormat("#,###.##");
}

Następnie zaimplementujmy `AccountAdapter`

In [ ]:
public class AccountsViewHolder extends RecyclerView.ViewHolder {

    private RecyclerviewItemViewBinding binding;

    public AccountsViewHolder(RecyclerviewItemViewBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(Account item){
        binding.RVNameTextView.setText(item.getName());
        binding.RVNumberTextView.setText(
                new StringBuilder(item.getNumber())
                        .replace(0, 6, "******").toString()
        );
        binding.RVValueTextView.setText(String.format(
                "%s zł",
                FormatterUtil.formatter.format(item.getAmount()))
        );
        binding.RVcolorBarView.setBackgroundColor(item.getColor());
    }
}

In [ ]:
public class AccountsAdapter extends RecyclerView.Adapter<AccountsViewHolder> {

    @NonNull
    @Override
    public AccountsViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new AccountsViewHolder(RecyclerviewItemViewBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull AccountsViewHolder holder, int position) {
        Account item = DataProvider.accounts[position];
        holder.bind(item);
    }

    @Override
    public int getItemCount() {
        return DataProvider.accounts.length;
    }
}

Do `AccountsFragment` dodajmy `ViewPager2` w metodzie `onViewCreated`

In [ ]:
public class AccountsFragment extends Fragment {

    private FragmentDetailInfoBinding binding;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentDetailInfoBinding.inflate(inflater);

        setupRecyclerView();

        return binding.getRoot();
    }

    private void setupRecyclerView() {
        binding.recyclerView.setAdapter(new AccountsAdapter());
        binding.recyclerView.setLayoutManager(new LinearLayoutManager(this.requireContext()));
    }
}

<img src="https://media2.giphy.com/media/grRVpWcSBxxdbt727W/giphy.gif?cid=790b76116c546b317dcc43461641e6ff4d09490235131cfa&rid=giphy.gif&ct=g" width="150" />

do layoutu `fragment_detail_info` dodajemy `DonutProgressView` i dwa pola `TextView`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<androidx.constraintlayout.widget.ConstraintLayout 
xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:tools="http://schemas.android.com/tools"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900"
    tools:context=".fragments.AccountsFragment">

        <TextView
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="@string/total"
            android:textColor="@color/white"
            android:textSize="26sp"
            app:layout_constraintBottom_toTopOf="@+id/totalAmountTextView"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent" />

        <TextView
            android:id="@+id/totalAmountTextView"
            android:layout_width="wrap_content"
            android:layout_height="wrap_content"
            android:text="100000 zł"
            android:textColor="@color/white"
            android:textSize="38sp"
            android:fontFamily="serif"
            app:layout_constraintBottom_toBottomOf="@+id/donut_viewAccount"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <app.futured.donut.DonutProgressView
            android:id="@+id/donut_viewAccount"
            android:layout_width="300dp"
            android:layout_height="300dp"
            app:donut_bgLineColor="@color/white"
            app:donut_gapAngle="20"
            app:donut_gapWidth="4"
            app:donut_animationDuration="1500"
            app:donut_strokeWidth="6dp"
            android:layout_marginTop="16dp"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintStart_toStartOf="parent"
            app:layout_constraintTop_toTopOf="parent" />

        <androidx.recyclerview.widget.RecyclerView
            android:id="@+id/recyclerView"
            android:layout_width="match_parent"
            android:layout_height="0dp"
            android:layout_marginTop="12dp"
            app:layout_constraintBottom_toBottomOf="parent"
            app:layout_constraintEnd_toEndOf="parent"
            app:layout_constraintTop_toBottomOf="@+id/donut_viewAccount" />


</androidx.constraintlayout.widget.ConstraintLayout>

W `AccountsFragment` dodajemy obsługę `DonutProgressView` - musimy przekazać dane jako listę `DonutSection`. `DonutSection` przyjmuje trzy argumenty
- `name` - nazwę
- `color` - kolor
- `amount` - wartość z przedziału 0.0 - 1.0

Do metody `OnCreateView` dodaję wywołania trzech metod konfigurujących elementy ui.

In [ ]:
override fun onCreateView(
    inflater: LayoutInflater, container: ViewGroup?,
    savedInstanceState: Bundle?
): View {
    binding = FragmentDetailInfoBinding.inflate(inflater)

    setupRecyclerView()
    setupDonutChart()
    setupTextViews()

    return binding.root
}

`setupRecyclerView` odpowiada za konfigurację `RecyclerView`

In [ ]:
private void setupRecyclerView() {
    binding.recyclerView.setAdapter(new AccountsAdapter());
    binding.recyclerView.setLayoutManager(new LinearLayoutManager(this.requireContext()));
}

`setupDonutChart` odpowiada za konfigurację `DonutChart`

In [ ]:
private void setupDonutChart() {
    List<DonutSection> values = new ArrayList<>();
    for (Account item : DataProvider.accounts){
        values.add(
                new DonutSection(
                        item.getName(),
                        item.getColor(),
                        ((float) item.getAmount() / (float) DataProvider.totalAccountsAmount))
        );
    }
    binding.donutViewAccount.submitData(values);
}

W widoku `binding.donutViewAccount` definiowane są wartości, które mają być przedstawione na wykresie. Na podstawie tych wartości tworzone są sekcje (klasa `DonutSection`), które zostaną wyświetlone na wykresie. Każda sekcja zawiera nazwę, kolor oraz wartość procentową związaną z kontem z listy `DataProvider.accounts`. W końcu, po utworzeniu sekcji, są one przekazywane do wykresu za pomocą funkcji `submitData(values)`, która rysuje wykres na podstawie dostarczonych danych.

`setupTextViews` - ustawia tekst pola `TextView`

In [ ]:
private void setupTextViews() {
    binding.totalAmountTextView.setText(String.format(
            "%s zł",
            FormatterUtil.formatter.format(DataProvider.totalAccountsAmount))
    );
}

W efekcie otrzymamy

<img src="https://media3.giphy.com/media/V6olKtypWuObTJXxmz/giphy.gif?cid=790b761116be30abedbc5c9f9c7a309c74243ac9e1060564&rid=giphy.gif&ct=g" width="150" />

### **BillsFragment**

Do `FormatterUtil` dodajmy `DateTimeFormatter` pozwalający sformatować datę

In [ ]:
val dateFormatter: DateTimeFormatter = DateTimeFormatter.ofPattern("yyyy MMM dd")

Przejdźmy do `BillAdapter`

In [ ]:
public class BillsViewHolder extends RecyclerView.ViewHolder {
    private final RecyclerviewItemViewBinding binding;

    public BillsViewHolder(RecyclerviewItemViewBinding binding) {
        super(binding.getRoot());
        this.binding = binding;
    }

    public void bind(Bill item){
        binding.RVNameTextView.setText(item.getName());
        binding.RVNumberTextView.setText(item.getDate().format(FormatterUtil.dateFormatter));
        binding.RVValueTextView.setText(String.format(
                "- %s zł",
                FormatterUtil.formatter.format(item.getAmount()))
        );
        binding.RVcolorBarView.setBackgroundColor(item.getColor());
    }
}

In [ ]:
public class BillsAdapter extends RecyclerView.Adapter<BillsViewHolder> {

    @NonNull
    @Override
    public BillsViewHolder onCreateViewHolder(@NonNull ViewGroup parent, int viewType) {
        return new BillsViewHolder(RecyclerviewItemViewBinding.inflate(
                LayoutInflater.from(parent.getContext()), parent, false
        ));
    }

    @Override
    public void onBindViewHolder(@NonNull BillsViewHolder holder, int position) {
        Bill item = DataProvider.bills[position];
        holder.bind(item);
    }

    @Override
    public int getItemCount() {
        return DataProvider.accounts.length;
    }

}

Korzystamy w tego samego layoutu co przy implementacji `AccountsFragment`

In [ ]:
public class BillsFragment extends Fragment {

    private FragmentDetailInfoBinding binding;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentDetailInfoBinding.inflate(inflater);

        setupRecyclerView();
        setupDonutChart();
        setupTextViews();

        return binding.getRoot();
    }

    private void setupTextViews() {
        binding.totalAmountTextView.setText(String.format(
                "%s zł",
                FormatterUtil.formatter.format(DataProvider.totalBillsAmount))
        );
    }

    private void setupDonutChart() {
        List<DonutSection> values = new ArrayList<>();
        for (Bill item : DataProvider.bills){
            values.add(
                    new DonutSection(
                            item.getName(),
                            item.getColor(),
                            ((float) item.getAmount() / (float) DataProvider.totalBillsAmount))
            );
        }
        binding.donutViewAccount.submitData(values);
    }

    private void setupRecyclerView() {
        binding.recyclerView.setAdapter(new BillsAdapter());
        binding.recyclerView.setLayoutManager(new LinearLayoutManager(this.requireContext()));
    }
}

### **OverviewFragment**

Layout `OverviewFragment` będzie się opierał na `CardView`.

Sam layout nie zawiera żadnych nowych elementów

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout xmlns:android="http://schemas.android.com/apk/res/android"
    xmlns:app="http://schemas.android.com/apk/res-auto"
    xmlns:tools="http://schemas.android.com/tools"
    android:layout_width="match_parent"
    android:layout_height="match_parent"
    android:background="@color/dark_blue_900"
    android:orientation="vertical"
    tools:context=".fragments.OverviewFragment">

     <!-- Alert section -->
    
    <androidx.cardview.widget.CardView
        android:id="@+id/alertCardView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:layout_marginTop="16dp"
        android:layout_marginStart="16dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical">

            <TextView
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:text="@string/alerts"
                android:textColor="@color/white"
                android:textSize="12sp" />

            <LinearLayout
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:orientation="horizontal">

                <TextView
                    android:layout_width="0dp"
                    android:layout_weight="1"
                    android:layout_height="wrap_content"
                    android:layout_marginStart="16dp"
                    android:text="@string/no_alerts"
                    android:textColor="@color/white"
                    android:textSize="24sp" />

                <Button
                    android:id="@+id/seeMoreButton"
                    android:layout_width="wrap_content"
                    android:layout_height="wrap_content"
                    android:backgroundTint="@android:color/holo_red_light"
                    android:textColor="@color/design_default_color_on_secondary"
                    android:layout_marginEnd="8dp"
                    android:text="@string/see_more" />
            </LinearLayout>
        </LinearLayout>
    </androidx.cardview.widget.CardView>

    <!-- AccountsSection -->
    
    <androidx.cardview.widget.CardView
        android:id="@+id/accouuntCardView"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginTop="8dp"
        android:layout_marginStart="16dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical"
            tools:context=".fragments.OverviewFragment">


            <TextView
                android:id="@+id/accountNameTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="8dp"
                android:text="@string/accounts"
                android:textColor="@color/white"
                android:textSize="12sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="parent" />

            <TextView
                android:id="@+id/accountTotalAmountTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginTop="4dp"
                android:gravity="center"
                android:text="11111111 zł"
                android:textColor="@color/white"
                android:textSize="36sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountNameTextView" />

            <androidx.recyclerview.widget.RecyclerView
                android:id="@+id/accountsRecyclerView"
                android:layout_width="match_parent"
                android:layout_height="0dp"
                android:layout_weight="1"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountTotalAmountTextView" />

            <Button
                android:id="@+id/accountsButton"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                android:text="@string/go_to_accounts"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/accountsRecyclerView"
                android:backgroundTint="@android:color/holo_red_light"
                android:textColor="@color/design_default_color_on_secondary"/>
        </LinearLayout>


    </androidx.cardview.widget.CardView>

    <androidx.cardview.widget.CardView <!-- Bills Section -->
        android:id="@+id/billsCardView"
        android:layout_width="match_parent"
        android:layout_height="0dp"
        android:layout_weight="1"
        android:layout_marginStart="16dp"
        android:layout_marginTop="8dp"
        android:layout_marginEnd="16dp"
        app:cardCornerRadius="12dp"
        app:cardElevation="15dp">

        <LinearLayout
            android:layout_width="match_parent"
            android:layout_height="match_parent"
            android:background="@color/dark_blue_500"
            android:orientation="vertical"
            tools:context=".fragments.OverviewFragment">


            <TextView
                android:id="@+id/billsNameTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="8dp"
                android:text="@string/bills"
                android:textColor="@color/white"
                android:textSize="12sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="parent" />

            <TextView
                android:id="@+id/billsTotalAmountTextView"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginTop="4dp"
                android:gravity="center"
                android:text="- 21999.99 zł"
                android:textColor="@color/white"
                android:textSize="36sp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toTopOf="@id/billsNameTextView" />

            <androidx.recyclerview.widget.RecyclerView
                android:id="@+id/billsRecyclerView"
                android:layout_width="match_parent"
                android:layout_height="0dp"
                android:layout_weight="1"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/billsTotalAmountTextView" />

            <Button
                android:id="@+id/billsButton"
                android:layout_width="match_parent"
                android:layout_height="wrap_content"
                android:layout_marginStart="16dp"
                android:layout_marginEnd="16dp"
                android:text="@string/go_to_bills"
                app:layout_constraintEnd_toEndOf="parent"
                app:layout_constraintStart_toStartOf="parent"
                app:layout_constraintTop_toBottomOf="@+id/billsRecyclerView"
                android:backgroundTint="@android:color/holo_red_light"
                android:textColor="@color/design_default_color_on_secondary"/>
        </LinearLayout>
    </androidx.cardview.widget.CardView>


</LinearLayout>

Do metody `onCreateView` dodajmy obsługę wszystkich elementów. Ponieważ nasz layout jest podzielony na trzy sekcje, dodajmy trzy funkcje pomocnicze.

In [ ]:
public class OverviewFragment extends Fragment {

    private FragmentOverviewBinding binding;

    @Override
    public View onCreateView(@NonNull LayoutInflater inflater, ViewGroup container,
                             Bundle savedInstanceState) {
        binding = FragmentOverviewBinding.inflate(inflater);

        setupAlertSection();
        setupAccountsSection();
        setupBillsSection();
        
        return binding.getRoot();
    }

    private void setupBillsSection() {
    }

    private void setupAccountsSection() {
    }

    private void setupAlertSection() {
    }
}

Rozpocznijmy od implementacji metody `setupAccountsSection`

In [ ]:
private void setupAccountsSection() {

    binding.accountsRecyclerView.setAdapter(new AccountsAdapter());
    binding.accountsRecyclerView.setLayoutManager(new LinearLayoutManager(this.requireContext()));

    binding.accountsButton.setOnClickListener(v -> {
        ViewPager2 viewPager2 = requireActivity().findViewById(R.id.viewPager);
        viewPager2.setCurrentItem(1);
    });

    binding.accountTotalAmountTextView.setText(String.format(
            "%s zł",
            FormatterUtil.formatter.format(DataProvider.totalAccountsAmount)));
}

Kolejno dodajmy obsługę dwóch przycisków - w tym przykładzie dodamy przejście na konkretną pozycję w `ViewPager2` - znajduje się on w głównej aktywności, więc z poziomu fragmentu nie mamy do niego dostępu. Możemy zastosować wykorzystując metod `requireActivity` oraz `findViewById`.

Alternatywnie możemy zapisać

In [ ]:
Activity hostingActivity = (MainActivity) requireActivity(); // musimy wykonać rzutowanie
hostingActivity.binding.viewpager.setCurrentItem(1);

Wtedy musimy zminić `binding` w `MainActivity` na `public`, oraz wykonać rzutowanie (`requireActivity` zwraca `FragmentActivity`, który nie posiada dostępu do właściwości `binding`). Chcąc pozbyć się zależności (trzymania twardej referencji do instancji `MainActivity` w instancji `OverviewFragment`) wykorzystuję uuuniwersalną metodę `findViewById` z klasy `FragemntActivity` - każda klasa dziedzicząca posiada tą metodę, więc nie muszę wykonywać rzutowania ani przechowywać referencji do instancji `MainActivity` w klasie `OverviewFragment`.

Następnie zaimplementujmy funkcję `setupBillsSection`, która będzie podobna do funkcji konfigurującej sekcję `accounts`

In [ ]:
private void setupBillsSection() {
    binding.billsRecyclerView.setAdapter(new BillsAdapter());
    binding.billsRecyclerView.setLayoutManager(new LinearLayoutManager(this.requireContext()));

    binding.billsButton.setOnClickListener(v -> {
        ViewPager2 viewPager2 = requireActivity().findViewById(R.id.viewPager);
        viewPager2.setCurrentItem(2);
    });

    binding.billsTotalAmountTextView.setText(String.format(
            "%s zł",
            FormatterUtil.formatter.format(DataProvider.totalBillsAmount)));
}

Ostatnim elementem będzie obsługa przycisku `SEE MORE` - tutaj wyświetlimy customowy `AlertDialog`. W pierwszsym kroku zdefiniujemy styl w pliku `themes.xml`

In [ ]:
<style name="MyDialogTheme" parent="Theme.AppCompat.Light.Dialog.Alert">
    <item name="android:background">@color/dark_blue_900</item>
    <item name="android:textColor">@color/white</item>
</style>

Następnie utwórzmy layout

`alert_dialog.xml`

In [ ]:
<?xml version="1.0" encoding="utf-8"?>
<LinearLayout
    xmlns:android="http://schemas.android.com/apk/res/android"
    android:orientation="vertical"
    android:paddingLeft="20dp"
    android:paddingRight="20dp"
    android:layout_width="match_parent"
    android:background="@color/dark_blue_900"
    android:layout_height="match_parent">

    <TextView
        android:id="@+id/alertTextView"
        android:layout_width="match_parent"
        android:layout_height="wrap_content"
        android:textSize="24sp"
        android:textColor="@color/white"
        android:gravity="center"
        android:text="@string/no_alerts"/>
</LinearLayout>

Na koniec przechodzimy do implementacji metody `setupAlertSection`

In [ ]:
private void setupAlertSection() {

    binding.seeMoreButton.setOnClickListener(v -> {
        new AlertDialog.Builder(getContext(), R.style.MyDialogTheme)
                .setTitle(getString(R.string.alerts)) // tytuł
                .setView(getLayoutInflater().inflate(R.layout.alert_dialog, null)) // layout
                .setPositiveButton("OK", (dialogInterface, i) -> {}) // domyślny przycisk pozwalający na wyjście
                .create() // utwórz AlertDialog
                .show(); // wyświetl Dialog
    });
}

Możemy przetestować aplikację.

<table><tr><td><img src="https://media1.giphy.com/media/49RzyXBmRBa59Wi47n/giphy.gif?cid=790b76111b9b9991bd4197bf8d7295e517603d04f8ba2992&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media1.giphy.com/media/zbqykvBYncTRQLCAja/giphy.gif?cid=790b7611d47e3d6d46e7b736ac29fe3cc9bc281aed07c1c0&rid=giphy.gif&ct=g" width="150" /></td><td><img src="https://media3.giphy.com/media/BbcDIUyV6IodX2yGVx/giphy.gif?cid=790b7611d1bf2abf7deacc8ad5a71a6a13adfbc9f4344954&rid=giphy.gif&ct=g" width="150" /></td></tr></table>